# 2. Obtención de datos

## MalagaHouse Scrapping

*Trabajo Fin de Máster FP en IA y Big Data realizado por Miguel Gámez Ruiz y Sergio Toscano Díaz*

### Descripción

Hemos decidido extraer los datos de viviendas que necesitamos de una página llamada [pisos.com](https://www.pisos.com/venta/pisos-malaga_capital_zona_urbana/).

Extraeremos los datos de las distintas zonas de Málaga Capital, para luego proceder al entrenamiento del modelo predictivo del precio de las viviendas.

### Código

Librerías:

In [ ]:
from bs4 import BeautifulSoup
import requests
from csv import writer
import re

Obtención de enlaces a scrapear:

In [ ]:
links = set()

# Extraigo enlaces de 74 paginaciones de la página web
for p in range(1,74):
    if p == 1:
        urls = "https://www.pisos.com/venta/pisos-malaga_capital_zona_urbana/"
    else:
        urls = "https://www.pisos.com/venta/pisos-malaga_capital_zona_urbana/" + str(p) + '/'

    page = requests.get(urls)

    soup = BeautifulSoup(page.content, 'html.parser')
    lonks = []

    for seccion in soup.findAll("a", {"class": "ad-preview__title"}):
        lonk = seccion.get("href")
        if "/comprar/" in lonk:
            link1 = "https://www.pisos.com" + lonk
            lonks.append(link1)

    links.update(lonks)

links = list(links)

Obtención de datos:

In [ ]:
with open('housing.csv', 'w', encoding='utf8', newline='') as f:
  thewriter = writer(f)
  header = ['Tipo', 'Zona', 'Precio', 'Superficie', 'Habitaciones', 'Baños', 'Garaje',
          'Trastero', 'Ascensor', 'Terraza', 'Amueblado', 'Chimenea', 'Piscina',
          'Jardín', 'Descripción']
  thewriter.writerow(header)
  
  for urls in links:
    page = requests.get(urls)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Extracción de los datos que vamos a utilizar y les creo variables por defecto
    datosbasicos = soup.findAll("li", {"class": "charblock-element more-padding"})
    equipamientos = soup.findAll("li", {"class": "charblock-element element-with-bullet"})

    sup = ""
    hab = ""
    ban = ""
    garaje = 0
    trastero = 0
    ascensor = 0
    terraza = 0
    amueblado = 0
    chimenea = 0
    piscina = 0
    jardin = 0
    ciudad_casa = ""
    tipo_casa = ""
    precio = 0

    # Obtención del precio
    precio_raw = soup.find('span', {"class": "h1 jsPrecioH1"})
    precio = precio_raw.text.strip() if precio_raw is not None else "0"
    precio = re.sub('[.€]', '', precio).strip()

    # Array de los datos
    todosdatosbasicos = []
    todosequipamientos = []

    # Obtención de Superficie, habitaciones y baños
    for datobasico in datosbasicos:
      datobasico = datobasico.text
      todosdatosbasicos.append(datobasico)

    for i in todosdatosbasicos:
      if "Superficie construida" in i:
        sup = i
      if "Habitaciones" in i:
        hab = i
      if "Baños" in i and ":"in i and not "Interior" in i:
        ban = i

    numeros = re.compile("(\d+)")
    if sup != '':
      superficie = [int(superficie) for superficie in str.split(sup) if superficie.isdigit()]
      if len(superficie) > 0:
        superficie = superficie[0]
      else:
        superficie = 0
    if hab != '':
      habitaciones = [int(habitaciones) for habitaciones in str.split(hab) if habitaciones.isdigit()]
      habitaciones = habitaciones[0]
    if ban != '':
        banyos = int(re.search('\d+', ban).group())

    # Obtención del equipamiento
    for equipamiento in equipamientos:
      equipamiento = equipamiento.text.strip()
      todosequipamientos.append(equipamiento)

    for i in todosequipamientos:
      if "Garaje" in i:
        garaje = 1
      elif "Trastero" in i:
        trastero = 1
      elif "Ascensor" in i:
        ascensor = 1
      elif "Terraza" in i:
        terraza = 1
      elif "Amueblado" in i:
        amueblado = 1
      elif "Chimenea" in i:
        chimenea = 1
      elif "Piscina" in i:
        piscina = 1
      elif "Jardín" in i:
        jardin = 1

    descripcion = soup.find('div', {"class": "description-container description-body"}).text.strip()

    # Obtencion del tipo de vivienda y la localización
    a_tipocasa = ['Casa', 'Piso', 'Chalet', 'Ático', 'Dúplex', 'Estudio', 'Finca rústica', 'Loft']

    a_zonas = ['Bailén-Miraflores', 'Campanillas', 'Carretera de Cádiz', 'Centro', 'Churriana', 
                  'Ciudad Jardín', 'Cruz de Humilladero', 'Este', 'La Rosaleda-La Roca', 'Puerto de la Torre', 
                  'Teatinos-Universidad']

    tipo_vivienda = soup.find('h1', {"class": "title"}).text.strip()
    localizacion = soup.find('h2', {"class": "position"}).text.strip()

    for tipo in a_tipocasa:
      if tipo in tipo_vivienda:
        tipo_casa = tipo

    for zona in a_zonas:
      if zona in localizacion:
        zona_casa = zona

    info = [tipo_casa, zona_casa, precio, superficie, habitaciones, banyos, garaje,
            trastero, ascensor, terraza, amueblado, chimenea, piscina, jardin,
            descripcion]
    thewriter.writerow(info)